In [33]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../data\ quality/quality.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 22:12:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 22:12:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [34]:
from pyspark.sql import functions as F

In [35]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [36]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ","
    reader = CSVReader(line_delimiter=",")
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df = client_code(reader, "../../../layer/bronze/educacao/IDESP_ESCOLA.csv")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [37]:
df.show()

+----------+----------+-----------+---------------+-------------+--------------------+-------------+-------------+-----------+------------+
|ANO_LETIVO|CODIGO_CIE|CODIGO_INEP|CodigoDiretoria|NomeDiretoria|          NomeEscola|NomeMunicipio|ANOS_INICIAIS|ANOS_FINAIS|ENSINO_MÉDIO|
+----------+----------+-----------+---------------+-------------+--------------------+-------------+-------------+-----------+------------+
|      2021|        12|   35000012|          10101|      NORTE 1|AYRES DE MOURA PR...|    SAO PAULO|         NULL|       3.55|        NULL|
|      2021|        24|   35000024|          10101|      NORTE 1|GAVIAO PEIXOTO BR...|    SAO PAULO|         3.11|       2.79|        1.69|
|      2021|        36|   35000036|          10101|      NORTE 1|        JOAO SOLIMEO|    SAO PAULO|         NULL|       2.79|        1.47|
|      2021|       772|   35000772|          10104|      NORTE 2|VISCONDE DE INHAU...|    SAO PAULO|         5.87|       NULL|        NULL|
|      2021|       7

In [38]:
# Instancia a classe de utilitários de qualidade de dados com o DataFrame original
dq = DataQualityUtils(df)

# Encadeia as transformações de qualidade de dados no DataFrame
df_tratado = (
    dq.remove_espacos_em_colunas()  # Remove espaços extras dos nomes das colunas
      .trim_strings()  # Remove espaços em branco no início e fim das strings de todas as colunas
      #.remover_caracteres_especiais(['localidades'])  # (Comentado) Remove caracteres especiais da coluna 'localidades'
      .remover_duplicatas()  # Remove registros duplicados do DataFrame
      .preencher_nulos({  # Preenche valores nulos com zero nas colunas especificadas
          'ANOS_INICIAIS': 0,
          'ANOS_FINAIS': 0,
          'ENSINO_MÉDIO': 0
      })
      .converter_para_numerico(['ANOS_INICIAIS', 'ANOS_FINAIS',
                                'ENSINO_MÉDIO'])
      .get_df()  # Retorna o DataFrame tratado
)


In [39]:
df_tratado.show()

+----------+----------+-----------+---------------+--------------------+--------------------+-------------------+-------------+-----------+------------+
|ANO_LETIVO|CODIGO_CIE|CODIGO_INEP|CodigoDiretoria|       NomeDiretoria|          NomeEscola|      NomeMunicipio|ANOS_INICIAIS|ANOS_FINAIS|ENSINO_MÉDIO|
+----------+----------+-----------+---------------+--------------------+--------------------+-------------------+-------------+-----------+------------+
|      2021|      8424|   35008424|          10603|         SANTO ANDRE|NELSON PIZZOTTI M...|        SANTO ANDRE|         3.41|       2.08|        1.45|
|      2021|     10546|   35010546|          10701|         CARAPICUIBA| JARDIM SANTA ANGELA|              COTIA|          3.1|       2.36|        1.17|
|      2021|     18156|   35018156|          20404|      CAMPINAS OESTE|AUREA ANUNCIACAO ...|           CAMPINAS|         3.54|       2.91|        2.62|
|      2021|     42523|   35042523|          20415|              SUMARE|LUIS HENRI

In [40]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df_tratado, "../../../layer/prata/educacao/idesp_escola/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")